I wrote this MCTS for tic tac toe game. The board, game, and MCTS algorithm is done now, and an interface with which you can play with AI! I got a draw for the first time.

Update: it's strange though the MCTS algorithm does not win any game, at all. Nov 1. I am updating.

# Monte Carlo Tree Search Tic-Tac-Toe

Housekeeping


Steps
1/ A board class to reflect the current state of the board, attributes and potential actions you can take on the boards.
  - input: actions on the nodes (changes on board state thereafter)
  - attributes: np array for current board states [2,3,3]
  - actions: change states, return results,
  - output: board states

2/ A node class to track the agents: what set of actions each has, how much rewards and resources they would have after taking certain actions.
  - inputs:
  - actions: move,
  - attributes: board states, pathes, rewards, resources,
  - output: final pathes

3/ MCTS function/class that implement the 4 steps of MCTS to execute TTT. Unclear about function/class for now. The reason that this is seperate from board/node is surely you can have different strategies for the same game. MCTS is only one of them.


Notice:



In [1]:
import numpy as np
import random
import unittest


In [2]:
# NEPCODE
# Board class
class Board(): # My general assumption is the states and constraints of the game come here.
  # def __init__(self, states = np.zeros([2,3,3])):  # NEP When you DIY a board state, as you will with rootstate, you will need to identify moves made by "1"s. By default it's supposed to be empty board.
  # NEP This header is wrong because  np.zeros([2,3,3]) is only evaluated once when the class is defined, not when every instance is defined.
  def __init__(self, states=None):
    self.states = np.zeros([2,3,3]) if states == None else states
    self.player = 0 # it starts with one player and this variable can be toggled to the other.

  # copy of *current* board states
  def copying(self):
    copy = Board() # surprising to me that you can define an instance in a method of a class. But here we go.
    copy.states = np.copy(self.states)
    copy.player = self.player  # suprising to me that they do not do a copy here. # NEP I just worried that they would be modified together.
    # Verify copy
    # print("Original unoccupied:", self.get_unoccupied())
    # print("Copy unoccupied:", copy.get_unoccupied())
    return copy

  # board_state updating
  def updating(self, position):  # position: (x,y) represents position to be updated
    row, col = position
    if np.any(self.states[:, row, col]):  # if cell is occupied
      print("Position", position, "is already occupied!")  # Debug: If occupied, report
      return  # position already taken
    self.states[self.player][row][col] = 1
    self.player ^= 1

  # get remaining moves (unoccupied) on the board
  def get_unoccupied(self):
    if self.resulting():  # We will not enter the function if result is declared.
      return []
    unoccupied =  np.argwhere(self.states[0] + self.states[1]==0).tolist() # both 0 --> np.any returns True --> np.argwhere returns indices of empty positions
    print("Unoccupied positions:", unoccupied)  # Debug: Show available moves
    return unoccupied

  def resulting (self): # function for returning the results (win/lose/draw)
    # NEP I suspect that this is not a function, because result should be automatically decided, not called. But it might be if it's called every time you make a move.
    # NEP updating: this should be written, because each time you run simulation you'd need to return a result (True/False), that can in return updates the nodes in BP.

    previous_player_board = self.states[self.player^1]  # You have to toggle back to last player who has moved to check their result. You do this because you toggle player each time you update the board.

    rows = np.any(np.sum(previous_player_board,axis=1)==3)  # NEP I am not too sure whether "any" would work fine here bc it's three rows to be "any-ed"
    columns = np.any(np.sum(previous_player_board,axis=0)==3)
    diagonals = np.any(np.trace(previous_player_board)==3)
    reverse_diagonals = np.any(np.trace(np.flip(previous_player_board, axis=1))==3)
    self.reward =  rows or columns or diagonals or reverse_diagonals  # if any suffice, we would return a True, which will send the signal back to the nodes in BP
    return self.reward


In [3]:
# NEPCODE
# Node class --> A kind of treenode tailored for TTT game
class Node():
  def __init__(self, parent = None, board= None, idx = None):
    self.parent = parent
    self.children = []   # NEP i initially defined this in header but it seems you cannot define it as a *mutable* list in the header.
    self.visits = 0
    self.wins = 0
    self.board = board
    self.idx = idx
    self.untried = board.get_unoccupied() if board is not None else [] # a list of indices pointing to untravelled nodes.
    # NEP I think in extension/simulation/a copy of current board state, untravelled=unvisited. Bc in that simulation, you'd actually make moves on board state as you would with actual board.
    # NEP here we added if board is not None else [] bc it runs into a scenario of board=None. But I do not think this should happen per how new node is initiated (always a board assigned)
  # make a selection  Conditions: highest UCB1 score  # NEP my problem with this is it seems UCB1 should be better in MCTS.
  def select(self):
    # A helper to ensure that the selected child is also the one with highest UCB1 score, which balances exploration and exploitation.
    for child in self.children:
        score = child.wins / child.visits + 0.2 * np.sqrt(2 * np.log(self.visits) / child.visits)
        print(f"Child {child.idx}: UCB1 score = {score}")

    s = sorted(self.children, key=lambda c:
               c.wins/c.visits + 0.2*np.sqrt(2*np.log(self.visits)/c.visits)
               if c.visits > 0 and self.visits > 0
               else float('inf')  # Unvisited nodes get infinite UCB1 value
               ) # NEP/GPT: Use of UCB1 requires defining a constant C for exploration, typically set to sqrt(2) in many MCTS implementations, rather than hardcoding 0.2.
    # if not s:
    #     return None  # or return random valid move
    return s[-1]  # the last of an ascending list, the next move of largest return.


  # connect a child to parent node. Input: current board, idx of untravelled node; Output: a connected child
  def expand(self, board, idx):
    child = Node(parent=self, board=board, idx=idx) # NEP to modify parent and board. They should be special to current child.
    self.untried.remove(idx)  # remove that from untravelled list.
    self.children.append(child) # Successfully extended the node into the list of children (to be selected)
    return child

  # for each node, there is win/visit num associated. Input: score from current run of MCTS
  def update(self, score):
    self.visits += 1
    self.wins += score


In [4]:
# NEPCODE
# MCTS # Specialized in MCTS strategy, not the game rules.

def MCTS(board=None, max_iter=None, idx=(-1, -1)):
  root = Node(board=board)  # current actor(node)
  iter = 0

  # each round is based on current board state/node
  while iter <= max_iter and len(board.get_unoccupied()) > 0:  # Conditions: smaller than max_iteration and still positions on board not tried.
                              # It was board.get_unoccupied() before, but an empty list is not None.
    iter += 1
    board = board.copying() # creating a new board copy when you start an iteration
    node = root # bc you will traverse to this copy of board state
    player = board.player  # When updating board state, the players will take turn.

    # Step 1 Selection: make a move / select a child on a copy of current states.
    while board.get_unoccupied() == [] and node.children != []: # Conditions: as long as the node is fully expanded and we have not reached terminal, we SELECT a child. Actually the more the better. You can pin down to the ternimal node by this approach. I mean why not it's based on UCB1 and it has all nodes as children.
      node = node.select()  # You traverse to best child on this copy state.
      board.updating(node.idx) #  NEP it troubles me here bc I did not see where possibly they have given an idx value to this node therefore you can just call it. But i guess it may just return None. In that case however, what's the point?
                                            # To answer my own question here. Yes it's defined. It's from self.children when SELECT. And any child idx is given and is randomly chosen fro untried moves.
    # Step 2 Expansion: the action of connecting to a untravelled node. Conditions: as long as there is untravelled position left
    if node.untried != []:  # if instead of why bc by setup you expand only one node a time.
      idx = random.choice(node.untried)
      board.updating(idx)
      node = node.expand(board=board.copying(), idx=idx) # Note that you create a new copy of board state for the child node that is seperate from current board state.

    # Step 3 Simulation: Going down from the newly expanded node to the terminal
    while board.get_unoccupied() !=[] and not board.resulting():
      board.updating(random.choice(board.get_unoccupied()))

    # Step 4 Backpropagation: return the stats to the parent node (how many steps and how many wins)
    while node:   # as long as there is a (parent) node we can come back to.
      won = board.resulting()
      if won:
        if board.player == player:  # NEP to figure out
          score = 1  # current player has won
        else:
          score = -1 # current player has lost
      else:
        score = 0  # draw
      node.update(score)
      node = node.parent

  sorted_nodes = sorted(root.children, key=lambda c: c.wins/c.visits)
  # Safety check
  if not sorted_nodes:
      return None  # or return random valid move
  return  sorted_nodes[-1].idx # the move with highest score.


### Unit tests - MCTS TTT

In [5]:


class TestBoard(unittest.TestCase):
    def setUp(self):
        self.board = Board()

        # Verify board is initialized correctly
        self.assertTrue(np.all(self.board.states == 0), "Board should start empty")

        # Verify we can get unoccupied positions
        unoccupied = self.board.get_unoccupied()
        self.assertIsNotNone(unoccupied, "Should get unoccupied positions")
        self.assertEqual(len(unoccupied), 9, "Empty board should have 9 positions")

    def test_initialization(self):
        self.assertTrue(np.array_equal(self.board.states, np.zeros((2, 3, 3))))
        self.assertEqual(self.board.player, 0)

    def test_copying(self):
        board_copy = self.board.copying()
        self.assertNotEqual(id(self.board.states), id(board_copy.states))
        self.assertEqual(self.board.player, board_copy.player)

    def test_updating(self):
        self.board.updating((1, 1))
        self.assertEqual(self.board.states[self.board.player ^ 1][1][1], 1)
        self.board.updating((1, 1))  # should do nothing since occupied
        self.assertEqual(self.board.states[self.board.player ^ 1][1][1], 1)

    def test_get_unoccupied(self):
        self.board.updating((1, 1))
        unoccupied = self.board.get_unoccupied()
        self.assertNotIn([1, 1], unoccupied)
        self.assertEqual(len(unoccupied), 8)

    def test_expansion(self):
        # Initial node setup with board
        self.node = Node(board=self.board)

        # Ensure untried moves exist
        self.assertTrue(len(self.node.untried) > 0,
                        f"No untried moves. Board state: {self.board.states}")

        idx = self.node.untried[0]
        new_child = self.node.expand(board=self.board.copying(), idx=idx)
        self.assertIn(new_child, self.node.children)
        self.assertNotIn(idx, self.node.untried)


class TestNode(unittest.TestCase):
    def setUp(self):
        self.board = Board()
        self.node = Node(board=self.board)

    def test_initialization(self):
        self.assertEqual(self.node.visits, 0)
        self.assertEqual(self.node.wins, 0)
        self.assertEqual(self.node.children, [])
        self.assertEqual(self.node.untried, self.board.get_unoccupied())

    def test_expansion(self):
        idx = self.node.untried[0]
        new_child = self.node.expand(board=self.board.copying(), idx=idx)
        self.assertIn(new_child, self.node.children)
        self.assertNotIn(idx, self.node.untried)

    def test_selection(self):
        # Initialize child nodes with mock idx and UCB1-related values
        child1 = Node(parent=self.node, idx=(0, 0), board=self.board)
        child2 = Node(parent=self.node, idx=(1, 1), board=self.board)

        child1.wins, child1.visits = 10, 20
        child2.wins, child2.visits = 5, 10

        self.node.children = [child1, child2]
        self.node.visits = 30  # total visits should be sum of children
        selected = self.node.select()

        self.assertEqual(selected, child2, "Expected the child with highest UCB1 score to be selected.")
    '''
    def test_selection(self):
      # Assuming self.node has children added and `child1` should have the highest UCB1 score
      selected = self.node.select()

      # Manually calculate expected highest UCB1 score for validation
      expected_scores = [
          child.win / child.visit + 0.2 * np.sqrt(2 * np.log(self.node.visit) / child.visit)
          for child in self.node.children
      ]
      highest_score_index = expected_scores.index(max(expected_scores))
      highest_score_child = self.node.children[highest_score_index]

      # Add a print statement to confirm selected node
      print(f"Expected highest child idx: {highest_score_child.idx}, Selected child idx: {selected.idx}")

      # Assert selected node matches expected highest score node
      self.assertEqual(selected, highest_score_child)
    '''

    def test_backpropagation(self):
        self.node.update(1)
        self.assertEqual(self.node.visits, 1)
        self.assertEqual(self.node.wins, 1)


class TestMCTS(unittest.TestCase):
    def setUp(self):
        self.board = Board()

    def test_mcts_execution(self):
        result = MCTS(board=self.board.copying(), max_iter=10)
        self.assertIsNotNone(result)

    def test_best_move_selection(self):
        best_move = MCTS(board=self.board.copying(), max_iter=100)
        self.assertIn(best_move, [idx for idx in self.board.get_unoccupied()])

    def test_simulated_game(self):
        board = Board()
        while not board.resulting():
            best_move = MCTS(board=board.copying(), max_iter=100)
            board.updating(best_move)
        self.assertTrue(board.resulting(), "Game should end with a result.")

if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


............
----------------------------------------------------------------------
Ran 12 tests in 0.194s

OK


Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Position (1, 1) is already occupied!
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions:

In [6]:


def play_game():
    # Initialize board and select max iterations for MCTS.
    board = Board()
    max_iter = 1000  # Adjust as necessary

    # Choose player symbols
    human_player = int(input("Choose your player (0 for Player 1, 1 for Player 2): "))
    ai_player = 1 - human_player  # AI is the other player

    print("\nStarting Tic-Tac-Toe Game!")
    print_board(board)

    # Game loop
    while not board.resulting() and board.get_unoccupied():
        if board.player == human_player:
            # Human move
            print("Your turn.")
            row, col = map(int, input("Enter your move as 'row col': ").split())
            if [row, col] in board.get_unoccupied():
                board.updating((row, col))
            else:
                print("Invalid move. Try again.")
                continue
        else:
            # AI move
            print("AI is making a move...")
            ai_move = MCTS(board.copying(), max_iter)
            if ai_move:
                board.updating(ai_move)
                print(f"AI chose position: {ai_move}")
            else:
                print("AI could not find a valid move.")
                break

        print_board(board)

    # Display result
    if board.resulting():
        winner = "You" if board.player == ai_player else "AI"
        print(f"Game Over! {winner} won!")
    else:
        print("Game Over! It's a draw.")

def print_board(board):
    display_board = np.full((3, 3), '-')
    for i in range(3):
        for j in range(3):
            if board.states[0][i][j] == 1:
                display_board[i][j] = 'X'  # Player 1
            elif board.states[1][i][j] == 1:
                display_board[i][j] = 'O'  # Player 2
    for row in display_board:
        print(" ".join(row))
    print()


In [ ]:
play_game()

Choose your player (0 for Player 1, 1 for Player 2): 0 

Starting Tic-Tac-Toe Game!
- - -
- - -
- - -

Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
Your turn.
Enter your move as 'row col': 1 1
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [2, 2]]
- - -
- X -
- - -

Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
AI is making a move...
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 0], [0, 1], [1, 0], [1, 2], [2, 0], [2, 1], [2, 2]]
Unoccupied positions: [[0, 1], [1, 